<a href="https://colab.research.google.com/github/mkaanerkoc/practices/blob/main/aave_liquidations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install web3
!pip install etherscan-python


In [ ]:
!pip install --force-reinstall protobuf==3.19.5

In [ ]:
from collections import (
    defaultdict,
    Counter,
)

import pandas as pd

import requests
import etherscan
from web3 import Web3
from etherscan import Etherscan

import json
import yaml

In [ ]:
ETHER_API_KEY = '################'
INFURA_KEY = 'https://mainnet.infura.io/v3/#####################'

LENDING_POOL = '0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9'
PROXY_CONTRACT = '0xC6845a5C768BF8D7681249f8927877Efda425baf'

eth_scan = Etherscan(ETHER_API_KEY)
w3 = Web3(Web3.HTTPProvider(INFURA_KEY))

proxy_contract_abi = eth_scan.get_contract_abi(address=PROXY_CONTRACT)
proxy_contract = w3.eth.contract(PROXY_CONTRACT, abi=proxy_contract_abi)

pool_contract_abi = eth_scan.get_contract_abi(address=LENDING_POOL)
pool_contract = w3.eth.contract(LENDING_POOL, abi=proxy_contract_abi)

In [ ]:
txs = eth_scan.get_normal_txs_by_address(address=LENDING_POOL, startblock=15506004, endblock=99999999, sort='asc')

In [ ]:
incoming_txs = list(filter(lambda x: x['to'] == LENDING_POOL.lower(), txs))

function_calls = Counter(map(lambda x: x['functionName'].split('(')[0], incoming_txs))
liquidation_txs = list(filter(lambda x: x['functionName'].find('liquidationCall') != -1, incoming_txs))
borrow_txs = list(filter(lambda x: x['functionName'].find('borrow') != -1, incoming_txs))
repay_txs = list(filter(lambda x: x['functionName'].find('repay') != -1, incoming_txs))

borrowers = list(map(lambda x: x['from'], borrow_txs))
repayers = list(map(lambda x: x['from'], repay_txs))

print(f'Total borrowing : {len(borrowers)}, Borrowers : {len(set(borrowers))}')
print(f'Total repaying : {len(repayers)}, Repayers : {len(set(repayers))}')

print(function_calls)

In [ ]:
borrow_tx_inputs = []

for borrow_tx in borrow_txs:
  func_obj, func_params = proxy_contract.decode_function_input(borrow_tx["input"])
  borrow_tx_inputs.append((borrow_tx['from'], func_params))

In [ ]:
differentBorrower = list(filter( lambda x: x[0].lower() != x[1]['onBehalfOf'].lower(), borrow_tx_inputs))

print(differentBorrower)
# assert all([borrower.lower() == args['onBehalfOf'].lower() for (borrower, args) in borrow_tx_inputs]), "borrower is not same as 'onBehalfOf' parameter"

In [ ]:
borrowers[0]

## Analyzing the liqudation calls

In [ ]:
liqudation_tx_inputs = []
for liq_tx in liquidation_txs:
  func_obj, func_params = proxy_contract.decode_function_input(liq_tx["input"])
  liqudation_tx_inputs.append((liq_tx['from'], func_params))

In [ ]:
liqudation_tx_inputs[0:10]

### Fetching borrower's account data by calling 'getUserAccountData' function

In [ ]:
borrower_account_data = []
for borrower in set(borrowers):
  res = pool_contract.functions.getUserAccountData(Web3.toChecksumAddress(borrower)).call()
  res.append(borrower)
  borrower_account_data.append(res)

In [ ]:
def convert_to_eth(wei):
  if isinstance(wei, str):
    return Web3.fromWei(int(wei), "ether")
  elif isinstance(wei, int):
    return Web3.fromWei(wei, "ether")

def process_account_data(account_data):
  account_data[0] = convert_to_eth(account_data[0])
  account_data[1] = convert_to_eth(account_data[1])
  account_data[2] = convert_to_eth(account_data[2])
  account_data[5] = convert_to_eth(account_data[5])
  return account_data

processed_account_data = list(map(process_account_data, borrower_account_data))


In [ ]:
dd = pd.DataFrame(processed_account_data, columns=['collateral', 'debt', 'available_borrow', 'liq_threshold', 'ltv', 'health_factor', 'address'])
dd['liq_threshold'] = dd['liq_threshold'] / 10000
dd['ltv'] = dd['ltv'] / 10000
dd.head()

In [ ]:
borrower_account_data = pd.read_csv('borrowers_account_info.csv', index_col=[0]).values.tolist()
borrower_account_data[0:2]

In [ ]:
dd.to_csv('borrowers_account_info.csv')

In [ ]:
bb = dd[dd['health_factor'] < 1.05]
bb = bb[bb['collateral']>0.5]
bb['debt_limit'] = bb['collateral'].astype(float) * bb['liq_threshold']
bb = bb.sort_values(by=['health_factor', 'collateral'], ascending=[True, False])

In [ ]:
bb